In [ ]:
import numpy as np
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from random import random, seed
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [1]:
# Make data.
n = 100
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
x, y = np.meshgrid(x,y)

x = x.ravel()
y = y.ravel()

def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)

MSE = np.zeros(3)
for ex_p in [0,1,2,3,4,5]:
    # we stack the values of x and y beside each other
    X = np.column_stack((x,y))

    # we use pipeline to fit our model
    pipeline = make_pipeline(
        PolynomialFeatures(degree=ex_p),
        StandardScaler(),
        Lasso(1e-4))

    pipeline.fit(X,z)
    z_predict = pipeline.predict(X)

    MSE[ex_p//5-1] = mean_squared_error(z_predict, z)

    x_plot = x.reshape((n, n))
    y_plot = y.reshape((n, n))
    z_plot = z.reshape((n, n))
    z_pred_plot = z_predict.reshape((n, n))


    fig = plt.figure(figsize=(12, 6))

    # Plotting the Franke function
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(x_plot, y_plot, z_plot, cmap='viridis', edgecolor='none')
    ax.set_title('Franke Function')

    # Plot the polynomial regression prediction
    ax = fig.add_subplot(122, projection='3d')
    ax.plot_surface(x_plot, y_plot, z_pred_plot, cmap='viridis', edgecolor='none')
    ax.set_title('Lasso Prediction')

    plt.show()

print(MSE)

NameError: name 'np' is not defined